Universidade Estadual de Campinas – UNICAMP 

Disciplina: Ciência e Visualização de Dados em Saúde

                    Análise de dados da hanseníase: uma abordagem preditiva para a saúde pública

# Imports e configs de módulos

In [55]:
import numpy as np
import pandas as pd
from IPython.display import Image
import warnings
from sklearn.feature_selection import f_classif, mutual_info_classif, SequentialFeatureSelector, SelectKBest
from scipy.stats import chi2_contingency
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, recall_score, precision_score, make_scorer, f1_score, accuracy_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut, cross_val_score, StratifiedKFold, GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from boruta import BorutaPy
from genetic_selection import GeneticSelectionCV
from hyperopt import fmin, tpe, hp
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from functools import partial
import xgboost as xgb
from lightgbm import LGBMClassifier


In [44]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 70)
%matplotlib inline
sns.set_style('darkgrid')
sns.set(font_scale=1.005)
warnings.filterwarnings('ignore')

# Datasets

In [45]:
df_hans_outcome = pd.read_csv('../data/processed/hans_outcome.csv')

In [46]:
df_hans_outcome.head()

,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_DIAG,ANO_NASC,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,SG_UF,ID_MN_RESI,ID_RG_RESI,ID_OCUPA_N,FORMACLINI,AVALIA_N,CLASSOPERA,MODODETECT,BACILOSCOP,DTINICTRAT,ESQ_INI_N,UFATUAL,ID_MUNI_AT,DT_NOTI_AT,ID_UNID_AT,UFRESAT,MUNIRESAT,CLASSATUAL,AVAL_ATU_N,ESQ_ATU_N,EPIS_RACIO,NU_ANO,NU_LESOES,CONTREG,DOSE_RECEB,CONTEXAM,DTALTA_N,TPALTA_N,DURACAO_TRAT
0,29.0,292700,1382.0,2653257.0,27/11/2008,1958.0,M,6.0,4.0,3.0,29.0,292700.0,1382.0,NaN,2.0,3.0,1.0,2.0,3.0,2009-03-02,1.0,BA,292700.0,01/01/2009,2653257.0,29.0,292700.0,1.0,3.0,1.0,4.0,2009,1.0,4.0,6.0,NaN,2009-12-29,Cura,302
1,21.0,210330,1434.0,2449706.0,01/01/2009,1943.0,M,6.0,4.0,1.0,21.0,210330.0,1434.0,NaN,4.0,1.0,2.0,1.0,1.0,2009-10-23,2.0,MA,210330.0,01/01/2009,2449706.0,21.0,210330.0,2.0,1.0,2.0,4.0,2009,10.0,6.0,12.0,1.0,2010-09-22,Cura,334
2,26.0,260290,1497.0,20389.0,13/10/2008,1994.0,F,9.0,NaN,NaN,26.0,260290.0,1497.0,999991.0,1.0,0.0,1.0,1.0,2.0,2008-10-13,1.0,PE,260290.0,02/01/2009,20389.0,26.0,260290.0,1.0,NaN,1.0,NaN,2009,1.0,3.0,NaN,3.0,2009-05-22,Cura,221
3,21.0,211290,1432.0,2455706.0,02/01/2009,1957.0,M,6.0,1.0,2.0,21.0,211290.0,1432.0,NaN,4.0,3.0,2.0,2.0,1.0,2009-01-02,2.0,MA,211290.0,02/01/2009,2455706.0,21.0,211290.0,2.0,3.0,2.0,4.0,2009,2.0,6.0,12.0,1.0,2010-04-29,Cura,482
4,35.0,354350,1575.0,2053381.0,18/12/2008,1968.0,M,6.0,1.0,1.0,35.0,354350.0,1575.0,NaN,1.0,0.0,2.0,1.0,1.0,2008-12-18,2.0,SP,354350.0,02/01/2009,2053381.0,35.0,354350.0,2.0,1.0,2.0,4.0,2009,1.0,2.0,NaN,2.0,2009-12-18,Cura,365


## ML Tests

### Feature Transformers

In [47]:
# Altera tipos de colunas
cols_to_change = [elem for elem in df_hans_outcome.columns if 'ID_' in elem or 'CS_' in elem or 'UF' in elem or 'MUNI' in elem]
others_cols = ['FORMACLINI', 'AVALIA_N', 'CLASSOPERA', 'MODODETECT', 'BACILOSCOP', 'ESQ_INI_N', 'CLASSATUAL', 
                'AVAL_ATU_N', 'ESQ_ATU_N', 'EPIS_RACIO', 'TPALTA_N']
cols_to_change = cols_to_change + others_cols
df_hans_outcome[cols_to_change] = df_hans_outcome[cols_to_change].astype(str)

In [48]:
target = 'TPALTA_N'
feats = df_hans_outcome.drop(target, axis=1).columns
num_feats = [feat for feat in feats if df_hans_outcome[feat].dtype != 'O']
cat_feats = [feat for feat in feats if feat not in num_feats]

In [49]:
# cat_feats = [feat for feat in cat_feats if 'DT' not in feat if target not in feat if 'ID' not in feat if 'ESCOL' not in feat]
feats_to_remove = ['SG_UF_NOT', 'CS_RACA', 'CS_ESCOL_N', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE',  'ID_OCUPA_N']
num_feats = [feat for feat in num_feats if 'DT' not in feat if 'AT' not in feat  if 'RESI' not in feat]
cat_feats = [feat for feat in cat_feats if 'DT' not in feat if 'AT' not in feat if 'RESI' not in feat if feat not in feats_to_remove]
cat_feats

['CS_SEXO',
 'CS_GESTANT',
 'SG_UF',
 'FORMACLINI',
 'AVALIA_N',
 'CLASSOPERA',
 'MODODETECT',
 'BACILOSCOP',
 'ESQ_INI_N',
 'EPIS_RACIO']

In [50]:
num_transformer = FeatureUnion(    
    [
        ('num_pipe', Pipeline(
            [
                ('norm', StandardScaler()),
                ('nan_input', SimpleImputer())
            ]
        )),
        ('nan_flag', MissingIndicator(error_on_new=False))
    ]
)
feat_transformer = ColumnTransformer(
    [
        ('num_trans', num_transformer, num_feats),
        ('cat_trans', OneHotEncoder(handle_unknown='ignore'), cat_feats)    
    ],
    remainder='passthrough', sparse_threshold=0
)

In [51]:
X = df_hans_outcome[num_feats + cat_feats].copy()
le = LabelEncoder().fit(df_hans_outcome['TPALTA_N'])
y = le.transform(df_hans_outcome['TPALTA_N'])

### Nested K-Fold

In [52]:
class NestedKFoldOpt():
    def __init__(self, ml_model, opt_space, loss_metric, outer_cv, inner_cv, 
                 opt_type='bayes', max_evals=10):
        self.ml_model = ml_model
        self.opt_space = opt_space
        self.loss_metric = loss_metric
        self.opt_type = opt_type
        self.max_evals = max_evals
        self.outer_cv = outer_cv
        self.inner_cv = inner_cv
        self.metrics_ = None
        self.metrics_oof_ = None
        self.metrics_dist_ = None
        self.best_hyperparameters_ = None
        
    def objective(self, x, data):
        model = clone(self.ml_model).set_params(**x)
        
        preds = cross_val_predict(model, data[0], data[1], cv=self.inner_cv, n_jobs=-1)
        
        return -self.loss_metric(data[1], preds)     
        
    
    def optimize(self, X, y):
        if self.opt_type == 'bayes':
            obj = partial(self.objective, data=(X, y))
            best = fmin(obj, space=self.opt_space, algo=tpe.suggest, 
                        max_evals=self.max_evals, return_argmin=False)
        else:
            loss_metric = make_scorer(self.loss_metric)
            best = GridSearchCV(self.ml_model, self.opt_space, scoring=loss_metric,
                                n_jobs=-1, cv=self.inner_cv, verbose=3).\
                   fit(X, y).best_params_
        return best
    
    def nested_kfold(self, X, y):
        recall_0 = []
        recall_1 = []
        precision_0 = []
        precision_1 = []
        oof = np.zeros(len(X))
        for train_idx, val_idx in self.outer_cv.split(X, y):
            X = pd.DataFrame(X)
            y = pd.Series(y)
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
            hypers = self.optimize(X_train, y_train)
            model = clone(self.ml_model).set_params(**hypers).fit(X_train, y_train)
            preds = model.predict(X_val)
            oof[val_idx] = preds 
            recall_0.append(recall_score(y_val, preds, pos_label=0))
            recall_1.append(recall_score(y_val, preds, pos_label=1))
            precision_0.append(precision_score(y_val, preds, pos_label=0))
            precision_1.append(precision_score(y_val, preds, pos_label=1))
        self.metrics_ = pd.DataFrame({'recall': [np.mean(recall_0), np.mean(recall_1)],
                                      'recall_std': [np.std(recall_0), np.std(recall_1)],
                                      'precision': [np.mean(precision_0), np.mean(precision_1)],
                                      'precision_std': [np.std(precision_0), np.std(precision_1)]},
                                     index=['class_0', 'class_1'])
        self.metrics_oof_ = pd.DataFrame({'recall': [recall_score(y, oof, pos_label=0), recall_score(y, oof)],
                                          'precision': [precision_score(y, oof, pos_label=0), precision_score(y, oof)],
                                          'f1 score': [f1_score(y, oof, pos_label=0), f1_score(y, oof)],
                                          'accuracy': [accuracy_score(y, oof)],
                                         },
                                         index=['class_0', 'class_1'])
        self.metrics_dist_ = {'recall_0': recall_0, 'recall_1': recall_1,
                              'precision_0': precision_0, 'precision_1': precision_1}
            
    def fit(self, X, y):
        X = X.copy()
        y = y.copy()
        self.nested_kfold(X, y)
        self.best_hyperparameters_ = self.optimize(X, y)

### Algorithms Comparison

#### Random Forest

In [109]:
rf_pipe = Pipeline([('feat_trans', feat_transformer),
                    ('over', SMOTE()),
                    ('rf', RandomForestClassifier())
                   ])

rf_opt_space = {'rf__criterion': hp.choice('rf__criterion', ['entropy', 'gini']),
         'rf__max_depth': hp.randint('rf__max_depth', 120),
         'rf__max_features': hp.choice('rf__max_features', ['auto', 'sqrt','log2', None]),
         'rf__min_samples_leaf': hp.uniform('rf__min_samples_leaf', 0, 0.5),
         'rf__min_samples_split' : hp.uniform ('rf__min_samples_split', 0, 1),
         'rf__n_estimators' : hp.choice('rf__n_estimators', [10, 50, 300, 750, 1200,1300,1500])}

rf_opt = NestedKFoldOpt(rf_pipe, rf_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(5, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(5, shuffle=True, random_state=9))

In [110]:
rf_opt.fit(X, y)
rf_opt.metrics_oof_

100%|██████████| 10/10 [54:31<00:00, 327.17s/trial, best loss: -0.6948070575169178] 


,recall,precision,f1 score,accuracy
class_0,0.825038,0.324768,0.466072,0.884123
class_1,0.887982,0.987296,0.935009,0.884123


In [79]:
preds = cross_val_predict(rf_pipe, X, y, cv=5)
print(classification_report(y, preds))

              precision    recall  f1-score   support

           0       0.63      0.54      0.58     18644
           1       0.97      0.98      0.97    285498

    accuracy                           0.95    304142
   macro avg       0.80      0.76      0.78    304142
weighted avg       0.95      0.95      0.95    304142



#### Logistic Regression

In [53]:
lr_pipe = Pipeline(
  [
      ('feat_trans', feat_transformer),
      ('over', SMOTE()),
      ('logreg', LogisticRegression(random_state = 0))
]
)

# lr_opt_space = {'logreg__solver': hp.choice('logreg__solver', ['liblinear', 'lbfgs']),
#                 'logreg__C': hp.loguniform('logreg__C', np.log(1e-5), np.log(100))}
               
lr_opt_space = {'logreg__warm_start' : hp.choice('logreg__warm_start', [True, False]),
                'logreg__fit_intercept' : hp.choice('logreg__fit_intercept', [True, False]),
                'logreg__tol' : hp.uniform('logreg__tol', 0.00001, 0.0001),
                'logreg__C' : hp.uniform('logreg__C', 0.05, 3),
                'logreg__solver' : hp.choice('logreg__solver', ['newton-cg', 'lbfgs', 'liblinear']),
                'logreg__multi_class' : 'auto',
                'logreg__class_weight' : 'balanced'}

lr_opt = NestedKFoldOpt(lr_pipe, lr_opt_space, partial(f1_score, average='macro'), 
                      outer_cv=StratifiedKFold(2, shuffle=True, random_state=9), 
                      inner_cv=StratifiedKFold(2, shuffle=True, random_state=9))

In [54]:
lr_opt.fit(X, y)
lr_opt.metrics_oof_

100%|██████████| 10/10 [06:26<00:00, 38.64s/trial, best loss: -0.7586174962545218]


,recall,precision,f1 score,accuracy
class_0,0.816241,0.425345,0.559259,0.921136
class_1,0.927985,0.987234,0.956693,0.921136


#### XGBoost

In [120]:
xgb_pipe = Pipeline(
  [
      ('feat_trans', feat_transformer),
      ('over', SMOTE()),
      ('xgb', xgb.XGBClassifier())
]
)

xgb_opt_space = {'xgb__max_depth':  hp.choice('xgb__max_depth', np.arange(1, 14, dtype=int)),
                'xgb__gamma': hp.uniform ('xgb__gamma', 1,9),
                'xgb__reg_alpha' : hp.quniform('xgb__reg_alpha', 40,180,1),
                'xgb__reg_lambda' : hp.uniform('xgb__reg_lambda', 0,1),
                'xgb__colsample_bytree' : hp.uniform('xgb__colsample_bytree', 0.5,1),
                'xgb__min_child_weight' : hp.quniform('xgb__min_child_weight', 0, 10, 1),
                'xgb__n_estimators': 180}

xgb_opt = NestedKFoldOpt(xgb_pipe, xgb_opt_space, partial(f1_score, average='macro'), 
                      outer_cv=StratifiedKFold(2, shuffle=True, random_state=9), 
                      inner_cv=StratifiedKFold(2, shuffle=True, random_state=9))

In [121]:
xgb_opt.fit(X, y)
xgb_opt.metrics_oof_

100%|██████████| 10/10 [32:21<00:00, 194.17s/trial, best loss: -0.8222220159939124]


,recall,precision,f1 score,accuracy
class_0,0.651148,0.677720,0.664168,0.959634
class_1,0.979779,0.977277,0.978526,0.959634


#### LightGBM

In [56]:
lgbm_pipe = Pipeline(
    [
        ('feat_trans', feat_transformer),
        ('over', SMOTE()),
        ('lgbm', LGBMClassifier())
    ]
)

lgbm_opt_space = {'lgbm__learning_rate': hp.loguniform('lgbm__learning_rate', np.log(0.001), np.log(0.5)),
                  'lgbm__reg_alpha': hp.loguniform('lgbm__reg_alpha', np.log(0.001), np.log(1)),
                  'lgbm__reg_lambda': hp.loguniform('lgbm__reg_lambda', np.log(0.001), np.log(1)),
                  'lgbm__subsample': hp.uniform('lgbm__subsample', 0.2, 1),
                  'lgbm__colsample_bytree': hp.uniform('lgbm__colsample_bytree', 0.2, 1),
                  'lgbm__min_child_samples': scope.int(hp.quniform('lgbm__min_child_samples', 1, 100, 1)),
                  'lgbm__num_leaves': scope.int(hp.quniform('lgbm__num_leaves', 2, 50, 1)),
                  'lgbm__subsample_freq': scope.int(hp.quniform('lgbm__subsample_freq', 1, 10, 1)),
                  'lgbm__n_estimators': scope.int(hp.quniform('lgbm__n_estimators', 100, 5000, 1))}

lgbm_opt = NestedKFoldOpt(lgbm_pipe, lgbm_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(2, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(2, shuffle=True, random_state=9))

In [57]:
lgbm_opt.fit(X, y)
lgbm_opt.metrics_oof_

100%|██████████| 10/10 [27:26<00:00, 164.65s/trial, best loss: -0.8228664676660344]


,recall,precision,f1 score,accuracy
class_0,0.617464,0.714853,0.662599,0.961452
class_1,0.983916,0.975239,0.979558,0.961452


**Escolha:** XGBoost